In [28]:
import os
import sklearn.metrics as metrics
import sklearn
import numpy as np
import pandas as pd
import pymongo
import pickle
import time
from pymongo import MongoClient
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [29]:
conection=MongoClient("mongodb://localhost:27017/")
db=conection.car
data=db.segment
df = pd.DataFrame(data.find({},{'_id':False}))
df

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 47.3+ KB


In [31]:
df['class'],class_names = pd.factorize(df['class'])

In [32]:
print(class_names)
print(df['class'].unique())

Index(['unacc', 'acc', 'vgood', 'good'], dtype='object')
[0 1 2 3]


In [33]:
df['buying'],_ = pd.factorize(df['buying'])
df['maint'],_ = pd.factorize(df['maint'])
df['doors'],_ = pd.factorize(df['doors'])
df['persons'],_ = pd.factorize(df['persons'])
df['lug_boot'],_ = pd.factorize(df['lug_boot'])
df['safety'],_ = pd.factorize(df['safety'])
df

,buying,maint,doors,persons,lug_boot,safety,class
0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,2,0
3,0,0,0,0,1,0,0
4,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...
1723,3,3,3,2,1,1,3
1724,3,3,3,2,1,2,2
1725,3,3,3,2,2,0,0
1726,3,3,3,2,2,1,3


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   buying    1728 non-null   int32
 1   maint     1728 non-null   int32
 2   doors     1728 non-null   int32
 3   persons   1728 non-null   int32
 4   lug_boot  1728 non-null   int32
 5   safety    1728 non-null   int32
 6   class     1728 non-null   int32
dtypes: int32(7)
memory usage: 47.3 KB


In [35]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]# split data randomly into 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [36]:
from sklearn.metrics import accuracy_score
for K in range(25):
    K_value = K+1
    neigh = KNeighborsClassifier(n_neighbors = K_value)
    neigh.fit(X_train, y_train)
    y_pred = neigh.predict(X_test)
    acc=accuracy_score(y_test,y_pred)*100
    print("Accuracy is ", acc,"% for K-Value:",K_value)
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["car"]
    mycol = mydb["acc"]

    mydict = { "Accuracy":acc , "kvalue": K_value }

    x = mycol.insert_one(mydict)

Accuracy is  83.62235067437379 % for K-Value: 1
Accuracy is  80.15414258188824 % for K-Value: 2
Accuracy is  89.21001926782274 % for K-Value: 3
Accuracy is  88.82466281310212 % for K-Value: 4
Accuracy is  93.83429672447014 % for K-Value: 5
Accuracy is  92.8709055876686 % for K-Value: 6
Accuracy is  92.8709055876686 % for K-Value: 7
Accuracy is  89.78805394990366 % for K-Value: 8
Accuracy is  90.94412331406551 % for K-Value: 9
Accuracy is  88.82466281310212 % for K-Value: 10
Accuracy is  89.40269749518305 % for K-Value: 11
Accuracy is  88.6319845857418 % for K-Value: 12
Accuracy is  88.82466281310212 % for K-Value: 13
Accuracy is  89.01734104046243 % for K-Value: 14
Accuracy is  89.78805394990366 % for K-Value: 15
Accuracy is  88.6319845857418 % for K-Value: 16
Accuracy is  88.82466281310212 % for K-Value: 17
Accuracy is  88.4393063583815 % for K-Value: 18
Accuracy is  88.6319845857418 % for K-Value: 19
Accuracy is  88.6319845857418 % for K-Value: 20
Accuracy is  88.2466281310212 % for 

In [37]:
x=mydb.acc.find_one(sort=[("Accuracy", pymongo.DESCENDING)])
y=x["kvalue"]
print(y)


5


In [38]:
# train the decision tree
## Instantiate the model with 5 neighbors.
model = KNeighborsClassifier(n_neighbors=y)
## Fit the model on the training data.
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [39]:

def save_model_to_db(model,client,db,dbconnection,model_name):
    pickled_model=pickle.dumps(model)
# creating connection
    myclient=pymongo.MongoClient(client)
#creating database in mongodb mydb myclient[db)
    mydb=myclient[db]
#creating collection
    mycon=mydb[dbconnection]
    info=mycon.insert_one({model_name: pickled_model, 'name': model_name, 'created_time':time.time()})
    print(info.inserted_id,"saved with this id successfullyt")
    details={
    'inserted_id':info.inserted_id,
    "model_name":model_name,
    'created_time':time.time()
    }
    return details

In [40]:
details=save_model_to_db(model=model,client='mongodb://localhost:27017/',db='car',dbconnection='data',model_name='carmodel')

5ee0a3d31ec3de364651c8ff saved with this id successfullyt


In [41]:
def load_saved_model_from_db(model_name,client,db,dbconnection):
    json_data={}
    myclient=pymongo.MongoClient(client)
#creating database in mongodb mydb myclient[db)
    mydb=myclient[db]
#creating collection
    mycon=mydb[dbconnection]
    data=mycon.find({'name':model_name})
    
    for i in data:
        json_data=i
    #fetching model from_db
    pickled_model=json_data[model_name]
    return pickle.loads(pickled_model)

In [42]:
x=load_saved_model_from_db(model_name=details['model_name'],client='mongodb://localhost:27017/',db='car',dbconnection='data')

In [43]:
print(x.score(X_test, y_test))

0.9383429672447013
